In [1]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-isekalala')

In [3]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [4]:
def ee_bytes_to_img(pixel_data):
    # no more num_channels check
    return np.stack([pixel_data[field] for field in pixel_data.dtype.names], axis=-1)

def geom_to_ee_fmt(geometry):
  # Convert the geometry to a GeoJSON format
  geometry_geojson = geometry.__geo_interface__

# Ensure it is in the format expected by ee.Geometry.MultiPolygon
  if geometry_geojson["type"] == "MultiPolygon":
    # Create an ee.Geometry.MultiPolygon
    ee_geometry = ee.Geometry.MultiPolygon(geometry_geojson["coordinates"])
  elif geometry_geojson["type"] == "Polygon":
    # If it's a single Polygon, convert it to MultiPolygon
    ee_geometry = ee.Geometry.MultiPolygon([geometry_geojson["coordinates"]])
  else:
    raise ValueError("Geometry type is not Polygon or MultiPolygon.")
  return ee_geometry

## Key Statistics for Electrification Planning

### Vegetation & Land Cover
- **NDVI mean, median, std**  
  → Proxy for biomass/agriculture; high NDVI areas may need different energy needs  
- **NDVI max–min (seasonality)**  
  → Indicates seasonal demand (e.g., irrigation pumping)  
- **EVI mean, std**  
  → Alternative vegetation index, less sensitive to saturation  
- **NDBI (built-up index) mean, std**  
  → Fraction of built environment; correlates with settlement density (OBV4)
- **MNDWI (water index) mean**  
  → Proximity to water bodies—affects microgrids, hydro-solar tradeoffs  

### Surface & Terrain
- **Elevation mean, std (SRTM)**  
  → High-terrain areas have different line-loss, infrastructure costs  
- **Slope mean, max**  
  → Steep slopes complicate distribution; matter for micro-hydro siting  

### Climate & Energy
- **Solar irradiance (GHI) mean**  
  → Directly drives PV potential (e.g., from the “NASA/POWER” dataset)  
- **Surface temperature (LST) mean, std**  
  → Hot areas increase cooling loads; thermal impact on panel efficiency  
- **Rainfall total (e.g., CHIRPS)**  
  → Seasonality of water supply & biomass; affects demand for irrigation pumps  

### Night-time Lights & Socio-economic Indicators
- **VIIRS nightlights mean**  
  → Proxy for existing electrification & economic activity  
- **Population density mean (WorldPop)**  
  → Demand proxy; ties into site prioritization  
- **Road density (km/km²)**  
  → Access and distribution cost proxy

### Observation Quality
- **Cloud-free days count**  
  → Data quality indicator: more cloud-free observations → more reliable stats
  
Extras might be: land-cover class proportions, soil moisture, vegetation height etc

In [5]:
# AOI GeoPackage
AOI_PATH = '/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/UGA Lamwo district.gpkg'
# Precomputed tile index GeoJSON
TILES_PATH = '/content/drive/Shareddrives/Sunbird AI/Projects/suntrace/suntrace-multimodal/data/lamwo_sentinel_composites/lamwo_grid.geojson'

# Year and bands
YEAR = 2023
NDVI_BANDS = ['B4', 'B8']
EVI_BANDS = ['B8','B4','B2']

# Earth Engine dataset IDs
datasets = {
    'S2_SR': 'COPERNICUS/S2_SR_HARMONIZED',
    'SRTM': 'USGS/SRTMGL1_003',
    'VIIRS': 'NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG',
    'MODIS_PAR': 'MODIS/062/MCD18C2',
    'OPEN_BUILDINGS': 'GOOGLE/Research/open-buildings-temporal/v1',
    'CHIRPS': 'UCSB-CHG/CHIRPS/DAILY',
    'WORLD_COVER': 'ESA/WorldCover/v200'
}

In [6]:
# Load AOI as GeoDataFrame
aoi_gdf = gpd.read_file(AOI_PATH)
aoi_geom = aoi_gdf.iloc[0].geometry

# Convert AOI to Earth Engine geometry
aoi_ee = geom_to_ee_fmt(aoi_geom)

# Load tile index as GeoDataFrame
tiles_gdf = gpd.read_file(TILES_PATH)

# Convert tile GeoDataFrame to GeoJSON and then EE FeatureCollection
import json

tiles_geojson = json.loads(tiles_gdf.to_json())
tiles_fc = ee.FeatureCollection(tiles_geojson)

In [7]:
# prompt: get the first element from tiles_fc

first_tile = tiles_fc.first()
print(first_tile.getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[32.20445276670657, 3.49577350008312], [32.19975686621689, 3.495769493085209], [32.200990009654106, 3.488396118256227], [32.20446032796104, 3.486788247328053], [32.20445276670657, 3.49577350008312]]]}, 'id': '0', 'properties': {}}


In [8]:
# Sentinel-2 filtered for cloud <20% and year
s2 = (
    ee.ImageCollection(datasets['S2_SR'])
      .filterBounds(aoi_ee)
      .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
)

# NDVI and EVI collections
ndvi_img = (
    s2.select(NDVI_BANDS)
      .map(lambda img: img.normalizedDifference(NDVI_BANDS).rename('NDVI'))
)
evi_img = (
    s2.select(EVI_BANDS)
      .map(lambda img: img.expression(
          '2.5*((B8 - B4)/(B8 + 6*B4 - 7.5*B2 + 1))',
          {'B8': img.select('B8'), 'B4': img.select('B4'), 'B2': img.select('B2')}
      ).rename('EVI'))
)

# Static terrain datasets
dem = ee.Image(datasets['SRTM'])
elevation = dem.select('elevation')
slope_img = ee.Terrain.slope(elevation)

# Open Buildings Temporal V1
ob_col = (
    ee.ImageCollection(datasets['OPEN_BUILDINGS'])
      .filterBounds(aoi_ee)
      .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
)
ob_mosaic = ob_col.mosaic()

# ESA WorldCover landcover image
wc = ee.ImageCollection(datasets['WORLD_COVER']).first()

In [ ]:
def compute_stats(tile):
    # Convert feature geometry to EE geometry
    geom = tile.geometry()
    proj = geom.projection()

    # Compute stats
    props = {}

    # --- Vegetation Indices (NDVI & EVI) ---
    ndvi_region = ndvi_img.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=30,
        bestEffort=True
    )
    props['ndvi_mean'] = ndvi_region.get('NDVI', ee.Number(-9999))

    ndvi_region = ndvi_img.median().reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=geom,
        scale=30,
        bestEffort=True
    )
    props['ndvi_med'] = ndvi_region.get('NDVI', ee.Number(-9999))

    # NDVI standard deviation (spatial mean of the temporal stdDev image)
    # Corrected reducer for spatial aggregation and corrected band name lookup
    ndvi_std_region = ndvi_img.reduce(ee.Reducer.stdDev()).reduceRegion(
        reducer=ee.Reducer.mean(), # Changed from median to mean for spatial aggregation
        geometry=geom,
        scale=30,
        bestEffort=True
    )
    # Corrected band name lookup - default name is original_band_name + '_stdDev'
    props['ndvi_std'] = ndvi_std_region.get('NDVI_stdDev', ee.Number(-9999))

    evi_region = evi_img.median().reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=geom,
        scale=30,
        bestEffort=True
    )
    props['evi_med'] = evi_region.get('EVI', ee.Number(-9999))

    # --- Terrain (Elevation & Slope) ---
    elev_region = dem.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=30,
        bestEffort=True
    )
    props['elev_mean'] = elev_region.get('elevation', ee.Number(-9999))

    slope_region = slope_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=30,
        bestEffort=True
    )
    props['slope_mean'] = slope_region.get('slope', ee.Number(-9999))

    # --- Solar PAR Mean ---
    par_region = ee.ImageCollection(datasets['MODIS_PAR']) \
        .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31") \
        .select('GMT_1200_PAR') \
        .mean() \
        .reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=1000,
            bestEffort=True
        )
    props['par_mean'] = par_region.get('GMT_1200_PAR', ee.Number(-9999))

    # --- Precipitation (CHIRPS) ---
    rain_col = ee.ImageCollection(datasets['CHIRPS']) \
        .filterBounds(geom) \
        .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31") \
        .select('precipitation')

    rain_sum = rain_col.sum().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=5566,
        bestEffort=True
    )
    props['rain_total_mm'] = rain_sum.get('precipitation', ee.Number(0))

    rain_mean = rain_col.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=5566,
        bestEffort=True
    )
    props['rain_mean_mm_day'] = rain_mean.get('precipitation', ee.Number(0))

    # --- Cloud-free Days Count ---
    cf_days = ee.ImageCollection(datasets['S2_SR']) \
        .filterBounds(geom) \
        .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31") \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .size()
    props['cloud_free_days'] = cf_days

    # --- Open Buildings Metrics ---
    props['bldg_count'] = ob_mosaic.reduceRegion(ee.Reducer.sum(), geometry=geom, scale=1, crs=proj) \
        .getNumber('building_fractional_count') \
        .multiply(ee.Number(2).pow(2))

    props['bldg_area'] = ob_mosaic.select('building_presence') \
        .gt(0.34) \
        .reduceRegion(ee.Reducer.sum(), geometry=geom, scale=1, crs=proj) \
        .getNumber('building_presence') \
        .multiply(ee.Number(1).pow(2))  # scale_m ** 2 to get areas in sqm

    bldg_height_stats = ob_mosaic.select('building_height').reduceRegion(reducer=ee.Reducer.minMax(), geometry=geom, scale=1, crs=proj)
    props['bldg_h_max'] = bldg_height_stats.get('building_height_max', ee.Number(0))

    # --- Land Cover Histogram ---
#    lc_hist = wc.select('Map') \
#        .reduceRegion(
#            reducer=ee.Reducer.frequencyHistogram(),
#            geometry=geom,
#            scale=10,
#            bestEffort=True
#        )
#    props['lc_hist'] = lc_hist.get('Map', ee.Dictionary({}))

    # Return feature with computed properties
    return tile.set(ee.Dictionary(props))

In [ ]:
# Map compute_stats over the EE FeatureCollection of tiles
stats_table = tiles_fc.map(compute_stats)

# Define the properties you want to export
# These should match the keys in the dictionary you set in compute_stats
selectors = [
    'ndvi_mean', 'ndvi_med', 'ndvi_std',
    'elev_mean', 'slope_mean',
    'par_mean', 'rain_total_mm', 'rain_mean_mm_day',
    'cloud_free_days',
    'bldg_count', 'bldg_area', 'bldg_h_max',
    # 'lc_hist', # If lc_hist is an EE Dictionary, it might require special handling or conversion
    'system:index' # Include the system index if you need a unique identifier
]

# Configure the export task
task = ee.batch.Export.table.toDrive(
    collection=stats_table,
    description='Lamwo_Tile_Stats_Export', # A name for your task
    folder='ee_exports', # Specify a folder in your Google Drive
    fileNamePrefix='Lamwo_Tile_Stats_EE',
    fileFormat='CSV',
    selectors=selectors
)

# Start the export task
task.start()

print('Earth Engine export task started. Check the Tasks tab in the Earth Engine Code Editor to monitor progress.')

Earth Engine export task started. Check the Tasks tab in the Earth Engine Code Editor to monitor progress.


In [19]:
stats_table.toList(10).getInfo()

[{'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[32.20445276670657, 3.49577350008312],
     [32.19975686621689, 3.495769493085209],
     [32.200990009654106, 3.488396118256227],
     [32.20446032796104, 3.486788247328053],
     [32.20445276670657, 3.49577350008312]]]},
  'id': '0',
  'properties': {'bldg_area': 0,
   'bldg_count': 0,
   'bldg_h_max': 0,
   'cf_days': 29,
   'elev_mean': 661.2470281065468,
   'evi_med': 1.9271495951115418,
   'ndvi_mean': -0.5076652662052633,
   'ndvi_med': -0.6625791733087613,
   'ndvi_std': 0.20529094747015791,
   'par_mean': 189.64341735839844,
   'rain_mean_mm_day': 3.2694852352142334,
   'rain_total_mm': 9.334060015397913,
   'slope_mean': 3.468728168306891}},
 {'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[32.204445134469694, 3.504819597219702],
     [32.201112174973126, 3.504816748300707],
     [32.198949922185626, 3.500594372249351],
     [32.19975686621689, 3.495769493085209],
     [32.2044

In [ ]:
# Map compute_stats over the EE FeatureCollection of tiles
stats_table = tiles_fc.map(compute_stats)
selectors = stats_table.first().propertyNames().getInfo()
explicit_selectors = [
    'NDVI_med', 'NDVI_std', 'EVI_med',
    'elev_mean', 'slope_mean',
    'par_mean', 'rain_total_mm', 'rain_mean_mm_day',
    'cf_days',
    'bldg_count', 'bldg_area', 'bldg_h_min', 'bldg_h_max',
    # 'lc_hist' is commented out, so don't include it here
    # If you uncomment lc_hist later, you might need to handle ee.Dictionary export
    # or convert it to a string/JSON string representation if GeoJSON export is tricky.
    'system:index' # Include the system index if you need a unique identifier
]

# Trigger export from Earth Engine
url = stats_table.getDownloadURL(**{
    'filetype': 'GeoJSON',
    'selectors': explicit_selectors
})

# Download the file to Colab environment
import requests
response = requests.get(url)

# Save locally
with open('/content/Lamwo_Tile_Stats_EE.geojson', 'wb') as f:
    f.write(response.content)

print('Download completed: /content/Lamwo_Tile_Stats_EE.geojson')

KeyboardInterrupt: 